In [5]:
def extract_material_faces(input_file, mirror_output_file, other_output_file, target_material="Glass"):
    vertices = []
    texture_coords = []
    normals = []
    mirror_faces = []
    other_faces = []
    current_material = None
    used_vertex_indices = set()
    used_texcoord_indices = set()
    used_normal_indices = set()

    with open(input_file, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue

        prefix = parts[0]

        # 记录顶点、纹理坐标和法线
        if prefix == 'v':
            vertices.append(line.strip())
        elif prefix == 'vt':
            texture_coords.append(line.strip())
        elif prefix == 'vn':
            normals.append(line.strip())

        # 检查材质名称
        elif prefix == 'usemtl':
            current_material = parts[1]

        # 分类面，保存到相应的列表
        elif prefix == 'f':
            if current_material == target_material:
                mirror_faces.append(parts[1:])
            else:
                other_faces.append(parts[1:])

            # 提取面中所有使用到的顶点、纹理坐标和法线的索引
            face = parts[1:]
            for vertex in face:
                indices = vertex.split('/')
                vertex_index = int(indices[0])
                used_vertex_indices.add(vertex_index)

                if len(indices) > 1 and indices[1]:
                    texcoord_index = int(indices[1])
                    used_texcoord_indices.add(texcoord_index)

                if len(indices) > 2 and indices[2]:
                    normal_index = int(indices[2])
                    used_normal_indices.add(normal_index)

    # 重新编号顶点、纹理坐标和法线索引
    vertex_index_mapping = {old_index: new_index for new_index, old_index in enumerate(sorted(used_vertex_indices), start=1)}
    texcoord_index_mapping = {old_index: new_index for new_index, old_index in enumerate(sorted(used_texcoord_indices), start=1)}
    normal_index_mapping = {old_index: new_index for new_index, old_index in enumerate(sorted(used_normal_indices), start=1)}

    # 构建新的顶点、纹理坐标和法线列表
    new_vertices = [vertices[old_index - 1] for old_index in sorted(used_vertex_indices)]
    new_texture_coords = [texture_coords[old_index - 1] for old_index in sorted(used_texcoord_indices)]
    new_normals = [normals[old_index - 1] for old_index in sorted(used_normal_indices)]

    # 更新面中的索引
    def update_faces(faces, vertex_mapping, texcoord_mapping, normal_mapping):
        new_faces = []
        for face in faces:
            new_face = []
            for vertex in face:
                indices = vertex.split('/')
                new_vertex_index = str(vertex_mapping[int(indices[0])])

                new_texcoord_index = indices[1]
                if len(indices) > 1 and indices[1]:
                    new_texcoord_index = str(texcoord_mapping[int(indices[1])])

                new_normal_index = indices[2]
                if len(indices) > 2 and indices[2]:
                    new_normal_index = str(normal_mapping[int(indices[2])])

                new_indices = [new_vertex_index]
                if len(indices) > 1:
                    new_indices.append(new_texcoord_index)
                if len(indices) > 2:
                    new_indices.append(new_normal_index)

                new_face.append('/'.join(new_indices))
            new_faces.append(f'{" ".join(new_face)}')
        return new_faces

    mirror_faces = update_faces(mirror_faces, vertex_index_mapping, texcoord_index_mapping, normal_index_mapping)
    other_faces = update_faces(other_faces, vertex_index_mapping, texcoord_index_mapping, normal_index_mapping)

    # 写入 mirror 材质的 obj 文件
    with open(mirror_output_file, 'w') as out:
        out.write("# Extracted OBJ file with lamp material\n")
        for vertex in new_vertices:
            out.write(f"{vertex}\n")
        for texcoord in new_texture_coords:
            out.write(f"{texcoord}\n")
        for normal in new_normals:
            out.write(f"{normal}\n")
        out.write(f"usemtl {target_material}\n")
        for face in mirror_faces:
            out.write(f"f {face}\n")

    # 写入其他材质的 obj 文件
    with open(other_output_file, 'w') as out:
        out.write("# Extracted OBJ file with other materials\n")
        for vertex in new_vertices:
            out.write(f"{vertex}\n")
        for texcoord in new_texture_coords:
            out.write(f"{texcoord}\n")
        for normal in new_normals:
            out.write(f"{normal}\n")
        out.write(f"usemtl other\n")
        for face in other_faces:
            out.write(f"f {face}\n")

# 示例用法
if __name__ == "__main__":
    input_file = r"D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\lattern_rough.obj"        # 输入的 OBJ 文件路径
    mirror_output_file = r"D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\lattern_glass.obj"  # 输出包含 mirror 材质的 OBJ 文件路径
    other_output_file = r"D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\other.obj"   # 输出其他材质的 OBJ 文件路径
    extract_material_faces(input_file, mirror_output_file, other_output_file)
    print(f"提取完成，镜面材质结果已保存到 {mirror_output_file}, 其他材质已保存到 {other_output_file}")


提取完成，镜面材质结果已保存到 D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\lattern_glass.obj, 其他材质已保存到 D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\other.obj


In [8]:
import numpy as np

def load_obj(file_path):
    """
    加载 .obj 文件中的顶点和面数据。
    
    :param file_path: .obj 文件路径
    :return: 顶点数组 (N, 3) 和面数组 (M, 3)
    """
    vertices = []
    faces = []
    
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.split()
            if len(parts) == 0:
                continue
            # 顶点数据
            if parts[0] == 'v':
                vertex = list(map(float, parts[1:4]))
                vertices.append(vertex)
            # 面数据
            elif parts[0] == 'f':
                face = [int(i.split('/')[0]) - 1 for i in parts[1:4]]  # 面索引从1开始，转换为0索引
                faces.append(face)
    
    return np.array(vertices), np.array(faces)


def save_obj(vertices, faces, file_path):
    """
    将顶点和面保存到 .obj 文件。
    
    :param vertices: 顶点数组 (N, 3)
    :param faces: 面数组 (M, 3)
    :param file_path: 保存的 .obj 文件路径
    """
    with open(file_path, 'w') as f:
        for vertex in vertices:
            f.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        for face in faces:
            f.write(f"f {face[0]+1} {face[1]+1} {face[2]+1}\n")  # 输出面索引从1开始


def remove_points_and_faces(vertices, faces, axis, threshold):
    """
    删除网格中指定坐标轴上大于某个值的顶点和面。
    
    :param vertices: 顶点数组 (N, 3)
    :param faces: 面数组 (M, 3)
    :param axis: 坐标轴，'x', 'y', 或 'z'
    :param threshold: 阈值，删除该坐标轴上大于此值的顶点和面
    :return: 删除顶点和面后的新网格 (新顶点，新面)
    """
    # 获取坐标轴对应的索引
    axis_index = {'x': 0, 'y': 1, 'z': 2}[axis]
    
    # 找出需要删除的顶点索引
    vertices_to_remove = set()
    for i, vertex in enumerate(vertices):
        if vertex[axis_index] > threshold:
            vertices_to_remove.add(i)
    
    # 保留没有被删除的顶点
    new_vertices = [vertex for i, vertex in enumerate(vertices) if i not in vertices_to_remove]
    
    # 创建一个映射，将旧顶点索引映射到新顶点索引
    vertex_mapping = {i: new_i for new_i, i in enumerate(range(len(vertices))) if i not in vertices_to_remove}
    
    # 删除面中涉及到已删除顶点的面
    new_faces = []
    for face in faces:
        # 如果面上包含删除的顶点，跳过
        if any(vertex_index in vertices_to_remove for vertex_index in face):
            continue
        # 否则，更新面中顶点的索引
        new_face = [vertex_mapping[vertex_index] for vertex_index in face]
        new_faces.append(new_face)
    
    # 返回新的顶点和面
    return np.array(new_vertices), np.array(new_faces)


# 示例：
input_obj = r'D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\lattern_glass.obj'  # 输入的.obj文件路径
output_obj = r'D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\lattern_new.obj'  # 输出的.obj文件路径

# 加载.obj文件
vertices, faces = load_obj(input_obj)

# 设置坐标轴和阈值
axis = 'z'      # 按照x坐标轴删除
threshold = 2.5  # 删除x大于2.5的顶点和面

# 调用函数删除顶点和面
new_vertices, new_faces = remove_points_and_faces(vertices, faces, axis, threshold)

# 将处理后的网格保存到新文件
save_obj(new_vertices, new_faces, output_obj)

print(f"处理后的网格已保存到 {output_obj}")


处理后的网格已保存到 D:\study\Computer Graphics\Sparkium-v2\assets\mesh\lattern\lattern_new.obj
